<a href="https://colab.research.google.com/github/Arseny20/robustLLM/blob/dataset/get_logs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U datasets
import pandas as pd
from transformers import pipeline
from tqdm import tqdm
import torch
import json
import re
from datasets import load_dataset


classifier = pipeline("zero-shot-classification",
                     model="MoritzLaurer/mDeBERTa-v3-base-mnli-xnli",
                     device=0 if torch.cuda.is_available() else -1)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 494.8/494.8 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 18.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.7.0
    Uninstalling fsspec-2025.7.0:
      Successfully uninstalled fsspec-2025.7.0
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.7.0 requires fsspec==2025.7.0, but you have fsspec 2025.3.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; pl

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/558M [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

spm.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/16.3M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/23.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/286 [00:00<?, ?B/s]

Device set to use cuda:0


In [2]:
# Login using e.g. `huggingface-cli login` to access this dataset
df = load_dataset("ai4privacy/pii-masking-400k")["train"]
df

README.md: 0.00B [00:00, ?B/s]

1en.jsonl:   0%|          | 0.00/84.8M [00:00<?, ?B/s]

de.jsonl:   0%|          | 0.00/82.7M [00:00<?, ?B/s]

es.jsonl:   0%|          | 0.00/42.5M [00:00<?, ?B/s]

fr.jsonl:   0%|          | 0.00/84.1M [00:00<?, ?B/s]

...

it.jsonl:   0%|          | 0.00/19.8M [00:00<?, ?B/s]

nl.jsonl: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/325517 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/81379 [00:00<?, ? examples/s]

Dataset({
    features: ['source_text', 'locale', 'language', 'split', 'privacy_mask', 'uid', 'masked_text', 'mbert_tokens', 'mbert_token_classes'],
    num_rows: 325517
})

In [3]:
def edit_json(input: str):
    json_dict = input
    json_new = list()
    for item in json_dict:
        label_mapping = {
            "PHONE": "Phone",
            "PASSWORD": "password",
            "USERNAME": "login",
            "EMAIL": "email",
            "GIVENNAME": "FIO",
            "SURNAME": "FIO"
        }
        original_label = item["label"]
        item["label"] = label_mapping.get(original_label, original_label)
        if item["label"] in ["Phone", "password", "login", "email", "FIO"]:
            json_new.append(item)
    return json_new


def edit_text(text: str):
    clean_text = re.sub(r'<[^>]+>', '', text) # <- Убираем HTML-теги

    html_entities = {
        '&nbsp;': ' ',
        '&amp;': '&',
        '&lt;': '<',
        '&gt;': '>',
        '&quot;': '"',
        '&apos;': "'"
    } # <- Замена сущностей с HTML

    for entity, replacement in html_entities.items():
        clean_text = clean_text.replace(entity, replacement)

    return clean_text.strip()
    return text

def word_to_ner(text, label_data):
    text = text.split()
    idx = 0
    new_list = list()
    mapping = ["Phone", "password", "login", "email", "FIO"]
    for word in text:
        if idx < len(label_data) and word == label_data[idx]["value"]:
            new_list.append(mapping.index(label_data[idx]["label"])+1)
            idx += 1
        else:
            new_list.append(0)
    return new_list


def tokens_marking(tokens, token_classes):
    token_mapping = {
        "B-PHONE": "B-Phone",
        "B-PASSWORD": "B-password",
        "B-USERNAME": "B-login",
        "B-EMAIL": "B-email",
        "B-GIVENNAME": "B-FIO",
        "B-SURNAME": "B-FIO",
        "I-PHONE": "I-Phone",
        "I-PASSWORD": "I-password",
        "I-USERNAME": "I-login",
        "I-EMAIL": "I-email",
        "I-GIVENNAME": "I-FIO",
        "I-SURNAME": "I-FIO"
    }

    token_classes = list(map(lambda x: token_mapping.get(x, 'O'), token_classes))

    result = []
    for token, cls in zip(tokens, token_classes):
        result.append((token, cls))

    return result

In [4]:
labels = [
    "development_logs",
    "application_logs",
    "corporate_logs",
    "other_logs",
    "message",
    "not_logs",
]


texts = list()
masks = list()
indices = list()
tokens = list()

for i in range(3000): # <- здесь вставить кол-во записей
    result = classifier(df[i]["source_text"], labels, multi_label=False)
    if result['labels'][0] in labels[0:3]:
        json_to_insert = edit_json(df[i]['privacy_mask'])
        if json_to_insert:
            print(f"{i:6}/{df.shape[0]} {result['labels'][0]:15} {df[i]['source_text'][:100]}...")
            print(json_to_insert)
            text_to_insert = df[i]['source_text'] # edit_text(df[i]['source_text']) <- убранные слова остаются в BIO-разметках, так что не стоит
            ner_indices = word_to_ner(text_to_insert, json_to_insert)
            tokens_to_insert = tokens_marking(df[i]['mbert_tokens'], df[i]['mbert_token_classes'])
            texts.append(text_to_insert)
            masks.append(json_to_insert)
            indices.append(ner_indices)
            tokens.append(tokens_to_insert)

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


   188/325517 application_logs Application completeness check: <br> Name: Marah Peyrilles <br> Job Title: Therapist, drama <br> ZIP...
[{'label': 'FIO', 'start': 43, 'end': 48, 'value': 'Marah', 'label_index': 1}, {'label': 'FIO', 'start': 49, 'end': 58, 'value': 'Peyrilles', 'label_index': 1}]
   315/325517 corporate_logs  Therapy check-in: Client Alfried Wohlgenannt, residing at White Springs, FL. Concerns over height (1...
[{'label': 'FIO', 'start': 25, 'end': 32, 'value': 'Alfried', 'label_index': 1}, {'label': 'FIO', 'start': 33, 'end': 44, 'value': 'Wohlgenannt', 'label_index': 1}]
   477/325517 development_logs The developmental evaluation of Niranjala Conejera focuses on their ability to engage socially and p...
[{'label': 'FIO', 'start': 32, 'end': 41, 'value': 'Niranjala', 'label_index': 1}, {'label': 'FIO', 'start': 42, 'end': 50, 'value': 'Conejera', 'label_index': 2}, {'label': 'FIO', 'start': 157, 'end': 165, 'value': 'Conejera', 'label_index': 1}]
   583/325517 developme

In [5]:
new_data = pd.DataFrame({
    "source_text": texts,
    "privacy_mask": masks,
    "ner_indices": indices,
    "tokens": tokens
})
new_data.to_csv('logs_data.csv', index=None)
new_data.head(10)

,source_text,privacy_mask,ner_indices,tokens
0,Application completeness check: <br> Name: Mar...,"[{'label': 'FIO', 'start': 43, 'end': 48, 'val...","[0, 0, 0, 0, 0, 5, 5, 0, 0, 0, 0, 0, 0, 0, 0, ...","[(Application, O), (complete, O), (##ness, O),..."
1,"Therapy check-in: Client Alfried Wohlgenannt, ...","[{'label': 'FIO', 'start': 25, 'end': 32, 'val...","[0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[(Therapy, O), (check, O), (-, O), (in, O), (:..."
2,The developmental evaluation of Niranjala Cone...,"[{'label': 'FIO', 'start': 32, 'end': 41, 'val...","[0, 0, 0, 0, 5, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[(The, O), (development, O), (##al, O), (evalu..."
3,Mrs Vlada's developmental delay noted. Residen...,"[{'label': 'FIO', 'start': 4, 'end': 9, 'value...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[(Mrs, O), (Vlad, B-FIO), (##a, I-FIO), (', O)..."
4,<p>fatima.yavru09's medication plan overseen b...,"[{'label': 'login', 'start': 3, 'end': 17, 'va...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[(<, O), (p, O), (>, O), (fat, B-login), (##im..."
5,2 Support Plan Document outlining indivi...,"[{'label': 'FIO', 'start': 67, 'end': 72, 'val...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[(2, O), (Support, O), (Plan, O), (Document, O..."
6,2. Export Compliance Report\nAssessment of the...,"[{'label': 'login', 'start': 112, 'end': 118, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[(2, O), (., O), (Export, O), (Com, O), (##pli..."
7,<html><body><p>Fund Performance: User jhdzvmhi...,"[{'label': 'login', 'start': 38, 'end': 57, 'v...","[0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[(<, O), (html, O), (>, O), (<, O), (body, O),..."
8,<html><body><p>Analysis of fund 1629729153 in ...,"[{'label': 'login', 'start': 113, 'end': 122, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[(<, O), (html, O), (>, O), (<, O), (body, O),..."
9,<html><body>2 Satellite Licensing Application ...,"[{'label': 'password', 'start': 238, 'end': 24...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[(<, O), (html, O), (>, O), (<, O), (body, O),..."
